# Masterclass: Web Scraping con Selenium

#### Cómo acceder a una web creando un bot con Selenium


A menudo, los datos están disponibles públicamente para nosotros, pero no en una forma que sea fácilmente utilizable. Ahí es donde entra en juego el Web Scraping, podemos usar web scraping para obtener nuestros datos deseados en un formato conveniente que luego se puede usar.

En este notebook mostraremos cómo se puede extraer información de interés de un sitio web usando la biblioteca Selenium.  

Selenium nos permite manejar una ventana del navegador e interactuar con el sitio web mediante programación con Python, además Selenium también tiene sus propias funciones y métodos que facilitan la extracción de datos.

Antes de comenzar os preguntaréis: Si ya he aprendido BeautifulSoup, ¿cuál es la diferencia con Selenium?

A diferencia BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un Document Object Model (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se verán reflejados en el DOM. 

Es decir, la estructura del proceso de web scraping con Selenium es la siguiente:

    URL → Solicitud HTTP → HTML → Selenium → DOM

¡Vamos allá!

## 1. Instalación de librerías

Tenemos que usar las librerías Selenium, Webdriver_manager y Undetected-chromedriver.  

*Nota: Undetected-chromedriver es necesaria para superar el Captcha de CloudFlare (cosa que no conseguimos solucionar en la sesión en directo).*  
*-> Solución aportada por Manuel Ros Martínez <-*

Vamos a instalarlas, descomenta la celda si es necesario:

In [1]:
# %pip install selenium
# %pip install webdriver_manager
# %pip install undetected-chromedriver

## 2. Importación de las librerías

In [1]:
# Para la manipulación de datos
import pandas as pd

# Servicio y driver de Chrome de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Las opciones que vamos a tener para buscar elementos
from selenium.webdriver.common.by import By

# Para cuando queramos mandar pulsaciones de teclado
from selenium.webdriver.common.keys import Keys

# Hacemos que espere
import time

# Importaciones para esperas explícitas (mejor práctica que time.sleep)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Importamos undetected-chromedriver para evitar el captcha
import undetected_chromedriver as uc

# Importamos excepciones comunes de Selenium para manejo de errores
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

## 3. Instalación del webdriver

El webdriver es lo que nos va a permitir conectarnos con el navegador.  

Lo instalamos y después lo creamos usando Undetected_chromedriver para evitar el captcha:

In [ ]:
service = Service(ChromeDriverManager().install())

# driver = webdriver.Chrome(service=service) # Este sería el webdriver estándar, no lo podemos usar para la web de Filmaffinity

In [3]:
driver = uc.Chrome(
        service=service,
        use_subprocess=False,
        headless=False,
    )

Después de ejecutar estas celdas debería haber aparecido en tu equipo un navegador web nuevo y vacío, ese navegador usa el webdriver comunicarse con el código que estamos ejecutando en este notebook.  

Dicho de otra forma, vamos a controlar el navegador usando código.

*Nota: en caso de tener problemas con el webdriver:*

>Para obtener una lista completa de controladores y plataformas compatibles, consulte [Selenium](https://www.selenium.dev/downloads/).  
>
>Si desea utilizar Google Chrome, diríjase a [chrome](https://chromedriver.chromium.org/) y descargue el controlador que corresponde a su >versión actual de Google Chrome.
>
>Para saber cual es la version de chrome que utilizo pegamos el siguiente enlace en la barra de chrome chrome://settings/help

## 4. Cargar la web
Cargamos la web en el driver para navegar y acceder a los elementos de la página.

In [4]:
url = "https://www.filmaffinity.com"
driver.get(url)

**IMPORTANTE:** Tras haber ejecutado la celda anterior ya deberías ver la web en el driver, **¡NO TOQUES NADA CON EL RATÓN!**, ahora empieza lo interesante.

## 5. Scrapeo

Podemos acceder a elementos en la página de varias maneras:  

- Nombre de la etiqueta
- Atributo: Clase
- Atributo: ID
- Atributo: Name
- Selector: Xpath
- Selector: CSS Selector

Para ello vamos a usar el driver que hemos creado y el By usando comandos como los siguientes:

>**Beginner Selenium Cheatsheet:**
>
>*Sacar un elemento*:
>- element = driver.find_element(by, value)
>
>*Sacar varios elementos*:
>- element = driver.find_elements(by, value)
>
>*Sacar atributos*:
>- attribute = element.--el atributo--
>- attribute = element.get_attribute(--el atributo--)
>
>*Hacer click*:
>- element.click()
>
>*Teclear*:
>- element.send_keys()
>
>*Gestión de pestañas*:
>- driver.switch_to.window(driver.window_handles[-1])
>- driver.get(url)
>- driver.close()

Perfecto, ahora vemos que hay un *pop-up* pidiendo que aceptemos las cookies, vamos a aceptarlo, **¡pero no lo hagas con el ratón!**, lo haremos con código.

Pero primero debemos encontrarlo, para eso necesitas:
1. Utilizar las herramientas de desarollador para inspeccionar los elementos en el HTML de la web.
2. Incluir lo que aprendas del HTML en tu código, como en los ejemplos que tenemos a continuación.

Nota: Recuerda que la estructura HTML de cada web es diferente, por lo que este proceso lleva tiempo.

### **Obtener todos los botones**

Tenemos diferentes formas de hacerlo, a continuación tienes 3 ejemplos:

In [5]:
# Encontrar elementos por etiqueta html
elements_by_tag = driver.find_elements(By.TAG_NAME, 'button')
print(type(elements_by_tag))

for i in elements_by_tag:
    print(i.text)

<class 'list'>
socios
MÁS OPCIONES
NO ACEPTO
ACEPTO


In [6]:
# Encontrar elementos por clase
element_by_class_name = driver.find_element(By.CLASS_NAME,'css-xlut8b')
print(element_by_class_name.find_element(By.TAG_NAME, 'span').text)

ACEPTO


In [7]:
# Encontrar elementos por XPATH
element_by_xpath = driver.find_element(By.XPATH, '//*[@id="accept-btn"]')
print(element_by_xpath.text)

ACEPTO


### **Encontrar el botón exacto**

In [8]:
#Primero, seleccionamos el botón, en este caso lo más preciso es usar su class_name
accept = driver.find_element(By.CLASS_NAME, 'css-xlut8b')

#Nos aseguramos de que es el botón que estamos buscando

print("Etiqueta: {}".format(accept.tag_name))
print("Texto de la etiqueta: {}".format(accept.text))
print("Atributo mode: {}".format(accept.get_attribute('mode')))
print("Atributo size: {}".format(accept.get_attribute('size')))

Etiqueta: button
Texto de la etiqueta: ACEPTO
Atributo mode: primary
Atributo size: large


In [9]:
#Hacemos click en el botón
accept.click()

### **Buscamos una película con la barra de búsqueda**

In [10]:
# Buscamos la barra de búsqueda de la página web por ID
search = driver.find_element(By.ID, 'top-search-input-2')
search

<undetected_chromedriver.webelement.WebElement (session="f0cd20e0e4f49270da2c65c62e14dd7c", element="f.B5EB59500092ECD354171AE003B4EFF2.d.DD40A54751630E110F49C6A7E83E9426.e.5")>

In [ ]:
# # Si necesitas verificar que es la barra de búsqueda también puedes hacer una captura del elemento
# search.screenshot('buscador.png')

True

In [12]:
# Teclea contenido dentro de la barra letra por letra (ejemplo educativo)
pelicula = 'Oblivion'

for letra in pelicula:
    search.send_keys(letra)
    time.sleep(0.1)  # Pequeña espera para simular escritura humana

In [14]:
# Limpiamos la barra
search.clear()

# Introducimos el nombre de la película a buscar de forma directa (más eficiente)
search.send_keys(pelicula)

# Usamos WebDriverWait para esperar a que aparezcan los resultados (mejor práctica que time.sleep)
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'movie-card-acf.select.hover')))

# Pulsamos intro para ir a la página de resultados
search.send_keys(Keys.ENTER)

### **Entramos en el primer resultado**

In [15]:
# Obtenemos el primer resultado de búsqueda
movie = driver.find_element(By.CLASS_NAME, 'se-it')

# Accedemos al enlace de la película mediante un selector CSS
url = movie.find_element(By.CSS_SELECTOR, 'div.mc-title a')

# Verificamos el título
print(f"Película encontrada: {url.text}")

Película encontrada: Oblivion


In [16]:
# Guardamos el enlace
link = url.get_attribute('href')
print(f"URL: {link}")

URL: https://www.filmaffinity.com/es/film618375.html


In [17]:
# Hacemos click en la película para entrar a su página
url.click()

### **Escrapeamos los datos de la película**

In [18]:
# Esperamos a que cargue la página de la película
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'movie-info')))

# Extraemos los datos principales de la película
data = driver.find_element(By.CLASS_NAME, 'movie-info')
dts = data.find_elements(By.TAG_NAME, 'dt')
dds = data.find_elements(By.TAG_NAME, 'dd')

In [19]:
# Comprobamos que el tamaño de dts y dds coincide
len(dts) == len(dds)

True

In [20]:
# Creamos la lista de columnas extrayendo los nombres de cada dt
lista_columnas = [col.text.strip() for col in dts]
lista_columnas

['Título original',
 'Año',
 'Duración',
 'País',
 'Dirección',
 'Guion',
 'Reparto',
 'Música',
 'Fotografía',
 'Compañías',
 'Género',
 'Sinopsis']

In [21]:
movie_dict = {col:[] for col in lista_columnas}

for indice, contenido in enumerate(dds):
    col = lista_columnas[indice]
    movie_dict[col].append(contenido.text.strip())

In [22]:
from pprint import pprint

pprint(movie_dict)

{'Año': ['2013'],
 'Compañías': ['Universal Pictures, Chernin Entertainment, Relativity Studios, '
               'Monolith Pictures, Radical Studios'],
 'Dirección': ['Joseph Kosinski'],
 'Duración': ['126 min.'],
 'Fotografía': ['Claudio Miranda'],
 'Guion': ['Joseph Kosinski, Michael Arndt, Karl Gajdusek. Cómic: Joseph '
           'Kosinski, Arvid Nelson'],
 'Género': ['Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. '
            'Cómic'],
 'Música': ['Anthony Gonzalez, M83, Joseph Trapanese'],
 'País': ['Estados Unidos'],
 'Reparto': ['Tom Cruise\n'
             'Andrea Riseborough\n'
             'Olga Kurylenko\n'
             'Morgan Freeman\n'
             'Nikolaj Coster-Waldau\n'
             'Zoe Bell'],
 'Sinopsis': ['Año 2073. Hace más de 60 años la Tierra fue atacada; se ganó la '
              'guerra, pero la mitad del planeta quedó destruido, y todos los '
              'seres humanos fueron evacuados. Jack Harper (Tom Cruise), un '
              'antig

### **Creamos un dataframe con los datos**

In [23]:
df = pd.DataFrame(movie_dict)

In [26]:
df

,Título original,Año,Duración,País,Dirección,Guion,Reparto,Música,Fotografía,Compañías,Género,Sinopsis
0,Oblivion,2013,126 min.,Estados Unidos,Joseph Kosinski,"Joseph Kosinski, Michael Arndt, Karl Gajdusek....",Tom Cruise\nAndrea Riseborough\nOlga Kurylenko...,"Anthony Gonzalez, M83, Joseph Trapanese",Claudio Miranda,"Universal Pictures, Chernin Entertainment, Rel...",Ciencia ficción. Intriga | Futuro postapocalíp...,Año 2073. Hace más de 60 años la Tierra fue at...


### **Limpiamos el dataframe**

Exploramos 'Reparto' y 'Género' que tienen cosas... raras

In [27]:
df["Reparto"][0]

'Tom Cruise\nAndrea Riseborough\nOlga Kurylenko\nMorgan Freeman\nNikolaj Coster-Waldau\nZoe Bell'

In [28]:
df["Género"][0]

'Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. Cómic'

Las limpiamos para arreglarlas

In [29]:
# Limpiamos el reparto: reemplazamos saltos de línea por comas
df["Reparto"] = df["Reparto"].str.replace("\n", ", ")

# Limpiamos el género: reemplazamos ". " (punto seguido de espacio) y " |" por comas
df["Género"] = df["Género"].str.replace(". ", ", ").str.replace(" |", ",")

In [30]:
df

,Título original,Año,Duración,País,Dirección,Guion,Reparto,Música,Fotografía,Compañías,Género,Sinopsis
0,Oblivion,2013,126 min.,Estados Unidos,Joseph Kosinski,"Joseph Kosinski, Michael Arndt, Karl Gajdusek....","Tom Cruise, Andrea Riseborough, Olga Kurylenko...","Anthony Gonzalez, M83, Joseph Trapanese",Claudio Miranda,"Universal Pictures, Chernin Entertainment, Rel...","Ciencia ficción, Intriga, Futuro postapocalípt...",Año 2073. Hace más de 60 años la Tierra fue at...


#### **¡Mucho mejor así! Y con esto hemos llegado al final de la Masterclass.**  

Ahora bien, si quisieras obtener datos de más películas... usa tus conocimientos para modificar el código de este cuaderno y poder escrapear su información poco a poco :)

## Extra:

### **Manejo de Errores (Buenas Prácticas)**

Es importante manejar excepciones al trabajar con Selenium, ya que los elementos pueden no estar disponibles o pueden cambiar. Las excepciones más comunes son:

- **NoSuchElementException**: El elemento no se encuentra en la página
- **TimeoutException**: El elemento no apareció en el tiempo esperado
- **ElementClickInterceptedException**: Otro elemento está bloqueando el click

In [ ]:
# ¡Esta celda SOLO contiene ejemplos! Si la ejecutas sin adaptar su código no obtendras resultado

# Ejemplo de manejo de errores al buscar un elemento
try:
    # Intentamos encontrar un elemento que podría no existir
    elemento = driver.find_element(By.ID, 'elemento-inexistente')
except NoSuchElementException:
    print("El elemento no se encontró en la página")


# Ejemplo con WebDriverWait y TimeoutException
try:
    wait = WebDriverWait(driver, 5)
    elemento = wait.until(EC.presence_of_element_located((By.ID, 'elemento-que-tarda')))
except TimeoutException:
    print("El elemento no apareció en el tiempo esperado")


# Ejemplo de ElementClickInterceptedException
try:
    boton = driver.find_element(By.ID, 'mi-boton')
    boton.click()
except ElementClickInterceptedException:
    print("Otro elemento está bloqueando el click, esperamos un momento")
    time.sleep(1)
    boton.click()  # Intentamos de nuevo


### **Comandos Útiles Adicionales**

Aquí tienes algunos comandos útiles para trabajar con ventanas y gestionar el ciclo de vida del driver:

**Abrir nueva ventana:**

In [32]:
driver.execute_script('window.open("");')

**Cambiar a otra ventana:**

In [33]:
driver.switch_to.window(driver.window_handles[1])

**Cerrar ventana actual:**

In [34]:
driver.close()

**Cerrar todo el driver:**

Es importante cerrar el driver cuando terminamos para liberar recursos. Una buena práctica es usar try-finally:

In [35]:
driver.quit()

In [ ]:
# Ejemplo de buena práctica: usar try-finally para asegurar el cierre del driver
# (Esto es solo un ejemplo educativo, no ejecutar)

"""
try:
    # Tu código de scraping aquí
    driver.get('https://www.filmaffinity.com')
    # ... más operaciones ...
finally:
    # Esto se ejecuta siempre, incluso si hay errores
    driver.quit()
"""